# Section 43: Foundations of Natural Language Processing

- Bonus Office Hours S.G.
- 10/27/20
- online-ds-ft-070620

> You are not allowed to use any propriety functions from this notebook in your projects. 

## Learning Objectives

- Discuss word Embeddings and their advantages
- Training Word2Vec models
- Using pretrained word embeddings


- Create a Classification Model for true-trump ("Twitter for Android") vs trump-staffer("Twitter for iPhone - from period of time when android was still in use)

    - Use lesson's W2Vec class in Sci-kit learn models
    - Use LSTMs
    - Use RNN/GRUs






- Compare:
    1.  Mean embeddings vs count/tfidf data with scikit learn.
    

## References

- My Work-in-Progress Capstone v2.0 Notebook:
    - [GitHub Notebook Link](https://github.com/jirvingphd/capstone-project-using-trumps-tweets-to-predict-stock-market/blob/WIP/Capstone%20Restarted%2010-2020.ipynb)

## NLP & Word Vectorization

> **_Natural Language Processing_**, or **_NLP_**, is the study of how computers can interact with humans through the use of human language.  Although this is a field that is quite important to Data Scientists, it does not belong to Data Science alone.  NLP has been around for quite a while, and sits at the intersection of *Computer Science*, *Artificial Intelligence*, *Linguistics*, and *Information Theory*. 

# Word Embeddings

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-word-embeddings-online-ds-ft-100719/master/images/embeddings.png">

- Convert words into a vector space
    + Mathematical object
- It's all about closeness
    + Distributional Hypothesis: https://en.wikipedia.org/wiki/Distributional_semantics#Distributional_hypothesis

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-word-embeddings-online-ds-ft-100719/master/images/vectors.png">

## Resources

- Kaggle Tutorial:  https://www.kaggle.com/learn/embeddings
- Google Embedding Crash Course: https://developers.google.com/machine-learning/crash-course/embeddings

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-using-word2vec-online-ds-ft-100719/master/images/training_data.png">

# Word2Vec

## Skip-Gram Model

- Train the MLP to find the best weights (context) to map word-to-word
- But since words close to another usually contain context, we're _really_ teaching it context in those weights
- Gut check: similar contexted words can be exchanged
    + EX: "A fluffy **dog** is a great pet" <--> "A fluffy **cat** is a great pet"

- By training a text-generation model, we wind up with a lookup table where each word has its own vector 

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-using-word2vec-online-ds-ft-100719/master/images/new_skip_gram_net_arch.png">


<img src="https://raw.githubusercontent.com/learn-co-students/dsc-using-word2vec-online-ds-ft-100719/master/images/new_word2vec_weight_matrix_lookup_table.png">

Each word will have a vector of contexts: the embeddings!

# Creating Word Embeddings with Trump's Tweets

In [1]:
!pip install -U fsds
from fsds.imports import *

fsds v0.2.27 loaded.  Read the docs: https://fs-ds.readthedocs.io/en/latest/ 


Handle,Package,Description
dp,IPython.display,Display modules with helpful display and clearing commands.
fs,fsds,Custom data science bootcamp student package
mpl,matplotlib,Matplotlib's base OOP module with formatting artists
plt,matplotlib.pyplot,Matplotlib's matlab-like plotting module
np,numpy,scientific computing with Python
pd,pandas,High performance data structures and tools
sns,seaborn,High-level data visualization library based on matplotlib


[i] Pandas .iplot() method activated.


In [2]:
df = pd.read_csv("../trump_tweets_12012016_to_01012020.csv")##'https://raw.githubusercontent.com/jirvingphd/capstone-project-using-trumps-tweets-to-predict-stock-market/master/data/trump_tweets_12012016_to_01012020.csv')
df['datetime'] = pd.to_datetime(df['created_at'])
df = df.set_index('datetime').sort_index()
df

,source,text,created_at,retweet_count,favorite_count,is_retweet,id_str
datetime,,,,,,,
2016-12-01 14:37:57,Twitter for iPhone,My thoughts and prayers are with those affecte...,12-01-2016 14:37:57,12077,65724,False,804333718999539712
2016-12-01 14:38:09,Twitter for Android,Getting ready to leave for the Great State of ...,12-01-2016 14:38:09,9834,57249,False,804333771021570048
2016-12-01 22:52:10,Twitter for iPhone,Heading to U.S. Bank Arena in Cincinnati Ohio ...,12-01-2016 22:52:10,5564,31256,False,804458095569158144
2016-12-02 02:45:18,Twitter for iPhone,Thank you Ohio! Together we made history – and...,12-02-2016 02:45:18,17283,72196,False,804516764562374656
2016-12-03 00:44:20,Twitter for Android,The President of Taiwan CALLED ME today to wis...,12-03-2016 00:44:20,24700,111106,False,804848711599882240
...,...,...,...,...,...,...,...
2020-01-01 01:17:43,Twitter for iPhone,RT @SenJohnKennedy: I think Speaker Pelosi is ...,01-01-2020 01:17:43,8893,0,True,1212181071988703232
2020-01-01 01:18:47,Twitter for iPhone,RT @DanScavino: https://t.co/CJRPySkF1Z,01-01-2020 01:18:47,10796,0,True,1212181341078458369
2020-01-01 01:22:28,Twitter for iPhone,Our fantastic First Lady! https://t.co/6iswto4WDI,01-01-2020 01:22:28,27567,132633,False,1212182267113680896


# Training Word2Vec

### Resources:

- Two Part Word2Vec Tutorial  (linked from Learn)
    - [Part 1: The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
    - [Part 2: Negative Sampling](http://mccormickml.com/2017/01/11/word2vec-tutorial-part-2-negative-sampling/)


- `sentences`: dataset to train on
- `size`: how big of a word vector do we want
- `window`: how many words around the target word to train with
- `min_count`: how many times the word shows up in corpus; we don't want words that are rarely used
- `workers`: number of threads (individual task "workers")

```python
from gensim.models import Word2Vec

# Let's assume we have our text corpus already tokenized and stored inside the variable 'data'--the regular text preprocessing steps still need to be handled before training a Word2Vec model!

model = Word2Vec(data, size=100, window=5, min_count=1, workers=4)

model.train(data, total_examples=model.corpus_count)
```

In [3]:
import re
test_str = df['text'].iloc[2]
print(test_str)
re.sub(r"(http[s]?://\w*\.\w*/+\w+)",' ',test_str)

Heading to U.S. Bank Arena in Cincinnati Ohio for a 7pm rally. Join me! Tickets: https://t.co/HiWqZvHv6M


'Heading to U.S. Bank Arena in Cincinnati Ohio for a 7pm rally. Join me! Tickets:  '

In [4]:
def find_urls(string): 
    return re.findall(r"(http[s]?://\w*\.\w*/+\w+)",string)
def sub_urls(string,replace_with=' '):
    return re.sub(r"(http[s]?://\w*\.\w*/+\w+)",replace_with,string)
df['text_clean'] = df['text'].apply(lambda x: sub_urls(x,' '))
df['text_clean'].iloc[2]

'Heading to U.S. Bank Arena in Cincinnati Ohio for a 7pm rally. Join me! Tickets:  '

In [5]:
df[df['text_clean'].str.contains('t.co')]['text'].values

array(['Boeing is building a brand new 747 Air Force One for future presidents but costs are out of control more than $4 billion. Cancel order!',
       'We cannot continue to let Israel be treated with such total disdain and disrespect. They used to have a great friend in the U.S. but.......',
       'RT @DanScavino: On behalf of our next #POTUS &amp; @TeamTrump-#HappyNewYear AMERICA🇺🇸https://t.co/Y6XDdijXeahttps://t.co/D8plL7xHqlhttps:…',
       '@CNN just released a book called "Unprecedented" which explores the 2016 race &amp; victory. Hope it does well but used worst cover photo of me!',
       'A beautiful funeral today for a real NYC hero Detective Steven McDonald. Our law enforcement community has my complete and total support.',
       'RT @MoskowitzEva: .@BetsyDeVos has the talent commitment and leadership capacity to revitalize our public schools and deliver the promise…',
       'Somebody with aptitude and conviction should buy the FAKE NEWS and failing @nytimes and either 

In [6]:
from nltk import word_tokenize
from gensim.utils import simple_preprocess
## TRAINING WORD2VEC FROM FULL DF NOT JUST TARGETS
data_lower = df['text_clean'].map(lambda x: simple_preprocess(x,True))#word_tokenize)

#  data_lower= list(map(lambda x: [w.lower() for w in x],data))

In [7]:
data_lower[2]

['heading',
 'to',
 'bank',
 'arena',
 'in',
 'cincinnati',
 'ohio',
 'for',
 'pm',
 'rally',
 'join',
 'me',
 'tickets']

In [8]:
from gensim.models import Word2Vec
# text = simple_preprocess(' '.join(df['text']))
# text

In [9]:
model = Word2Vec(data_lower,size=100,window=4,min_count=1,workers=4)

In [10]:
model.train(data_lower, total_examples=model.corpus_count,epochs=10)

(2777847, 3507490)

In [11]:
wv = model.wv

In [14]:
help(wv)

Help on Word2VecKeyedVectors in module gensim.models.keyedvectors object:

class Word2VecKeyedVectors(WordEmbeddingsKeyedVectors)
 |  Mapping between words and vectors for the :class:`~gensim.models.Word2Vec` model.
 |  Used to perform operations on the vectors such as vector lookup, distance, similarity etc.
 |  
 |  Method resolution order:
 |      Word2VecKeyedVectors
 |      WordEmbeddingsKeyedVectors
 |      BaseKeyedVectors
 |      gensim.utils.SaveLoad
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  save_word2vec_format(self, fname, fvocab=None, binary=False, total_vec=None)
 |      Store the input-hidden weight matrix in the same format used by the original
 |      C word2vec-tool, for compatibility.
 |      
 |      Parameters
 |      ----------
 |      fname : str
 |          The file path used to save the vectors in
 |      fvocab : str, optional
 |          Optional file path used to save the vocabulary
 |      binary : bool, optional
 |          If True, th

In [15]:
wv['republican']

array([-1.97725177e-01, -1.02185011e+00, -1.69189334e-01, -1.37412810e+00,
        1.47666514e+00, -3.74030292e-01,  2.40638542e+00,  1.76545548e+00,
       -9.86887336e-01, -1.50291061e+00, -1.17016053e+00, -1.30776596e+00,
        8.20626020e-02,  1.87480831e+00,  4.31128830e-01,  4.67176348e-01,
       -1.17880784e-01,  2.19573164e+00,  1.59704947e+00, -1.32507336e+00,
        1.30375671e+00, -8.85268748e-01,  7.43142426e-01, -1.30962408e+00,
        6.65495172e-02,  5.80909431e-01, -2.88448781e-01,  4.66280580e-01,
       -2.40665030e+00, -1.72075272e-01,  7.96539605e-01, -3.48868757e-01,
       -1.45647514e+00, -8.85340095e-01,  1.95581961e+00,  6.16921604e-01,
       -1.81026196e+00, -1.85274065e+00,  1.30017591e+00, -1.50146902e-01,
       -1.31920803e+00, -1.49311972e+00,  5.02608232e-02, -1.39201117e+00,
       -2.47322046e-03, -2.25241327e+00,  5.77637494e-01,  7.56581903e-01,
       -4.66473192e-01,  1.29378021e+00, -8.13583314e-01, -5.35659850e-01,
       -2.22131801e+00, -

In [16]:
wv.most_similar('republican')

[('democrat', 0.7401123642921448),
 ('senators', 0.7057304382324219),
 ('party', 0.7018539905548096),
 ('democratic', 0.6777942776679993),
 ('candidates', 0.6435932517051697),
 ('thedemocrats', 0.6297471523284912),
 ('opposition', 0.6088434457778931),
 ('bitterness', 0.6044570803642273),
 ('senjohnkennedy', 0.595993161201477),
 ('republicans', 0.5892543792724609)]

In [17]:
wv.most_similar(negative=['democrat'])

[('lifeline', 0.4959105849266052),
 ('skorea', 0.4835852384567261),
 ('nate', 0.4791489541530609),
 ('breakthru', 0.4329405725002289),
 ('headway', 0.38289856910705566),
 ('survive', 0.3765192925930023),
 ('rexnord', 0.3763708770275116),
 ('charismatic', 0.37339574098587036),
 ('proposing', 0.368845134973526),
 ('japan', 0.3668920397758484)]

#### Word2Vec params

```python
## For initializing model
sentences=None,
    size=100,
    alpha=0.025,
    window=5,
    min_count=5,
    max_vocab_size=None,
    sample=0.001,
    seed=1,
    workers=3,
    min_alpha=0.0001,
    sg=0,
    hs=0,
    negative=5,
    cbow_mean=1,
    hashfxn=<built-in function hash>,
    iter=5,
    null_word=0,
    trim_rule=None,
    sorted_vocab=1,
    batch_words=10000,
    compute_loss=False,
    callbacks=(),
    
    
## For training 
    sentences,
    total_examples=None,
    total_words=None,
    epochs=None,
    start_alpha=None,
    end_alpha=None,
    word_count=0,
    queue_factor=2,
    report_delay=1.0,
    compute_loss=False,
    callbacks=(),
    ```

In [19]:
# ### USING WORD VECTOR MATH TO GET A FEEL FOR QUALITY OF MODE
def word_math(wv,pos_words=['hillary'],neg_words=['bill'],
              verbose=True,return_vec=False):
    if isinstance(pos_words,str):
        pos_words=[pos_words]
    if isinstance(neg_words,str):
        neg_words=[neg_words]


    pos_eqn = '+'.join(pos_words)
    neg_eqn = '-'.join(neg_words)

    print('---'*15)    
    print(f"[i] Result for:\t{pos_eqn}{' - '+neg_eqn if len(neg_eqn)>0 else ' '}")
    print('---'*15)

    answer = wv.most_similar(positive=pos_words,negative=neg_words)
    
    if verbose:
          [print(f"- {ans[0]} ({round(ans[1],3)})") for ans in answer]
          print('---'*15,'\n\n')

    if return_vec: 
          return answer
    

In [20]:
equation_list=[(['america','crime'],[]),
               
               (['democrats','russia'],[]),
               (['republican'],['honor']),
               (['man','power'],[]),
               (['russia','honor'],[]),
              (['china','tariff'])]

for eqn in equation_list:
#     print('\n\n')
    word_math(wv,*eqn)
#     word_math(wv2,*eqn)

---------------------------------------------
[i] Result for:	america+crime 
---------------------------------------------
- borders (0.736)
- strong (0.652)
- choice (0.625)
- trajectory (0.622)
- tough (0.6)
- bracing (0.598)
- ussgeraldrford (0.597)
- agenda (0.594)
- vamissionact (0.589)
- causing (0.583)
--------------------------------------------- 


---------------------------------------------
[i] Result for:	democrats+russia 
---------------------------------------------
- dems (0.752)
- facts (0.732)
- hoax (0.716)
- russian (0.711)
- russians (0.687)
- dnc (0.674)
- phony (0.66)
- collusion (0.658)
- questions (0.657)
- rigged (0.655)
--------------------------------------------- 


---------------------------------------------
[i] Result for:	republican - honor
---------------------------------------------
- democrat (0.658)
- dem (0.563)
- dems (0.527)
- dre (0.51)
- republicans (0.497)
- democratic (0.46)
- liberal (0.454)
- veered (0.443)
- relying (0.442)
- quitters (0

___ 

___

## GloVe - Global Vectors for Word Representation

### Transfer Learning

- Usually embeddings are hundreds of dimensions
- Just use the word embeddings already learned from before!
    + Unless very specific terminology, context will likely carry within language
- Comparable to CNN transfer learning

# Sequence Models - ANNs 

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-sequence-model-use-cases-online-ds-ft-100719/master/images/rnn.gif">

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-understanding-recurrent-neural-networks-online-ds-ft-100719/master/images/unrolled.gif">

<img src="https://raw.githubusercontent.com/jirvingphd/dsc-sequence-model-use-cases-online-ds-ft-100719/master/images/RNN-unrolled.png">

### LSTMs GRUs

- GRU (Gated Recurrent Units (GRUs)
    - Reset Gate
    - Update Gate
    
- LSTM (Long Short Term Memory Cells)
   - Input Gate
   - Forget Gate
   - Output Gate

### ___

## Using Embeddings in Classification

### Embedding Layers
You should make note of a couple caveats that come with using embedding layers in your neural network -- namely:

* The embedding layer must always be the first layer of the network, meaning that it should immediately follow the `Input()` layer 
* All words in the text should be integer-encoded, with each unique word encoded as it's own unique integer  
* The size of the embedding layer must always be greater than the total vocabulary size of the dataset! The first parameter denotes the vocabulary size, while the second denotes the size of the actual word vectors
* The size of the sequences passed in as data must be set when creating the layer (all data will be converted to padded sequences of the same size during the preprocessing step) 


[Keras Documentation for Embedding Layers](https://keras.io/layers/embeddings/).

### Using Pre-Trained Vectors

In [ ]:
import os
folder = '/Users/jamesirving/Datasets/'#glove.twitter.27B/'
# print(os.listdir(folder))
glove_file = folder+'glove.6B/glove.6B.50d.txt'#'glove.twitter.27B.50d.txt'
glove_twitter_file = folder+'glove.twitter.27B/glove.twitter.27B.50d.txt'
print(glove_file)
print(glove_twitter_file)

#### Keeping only the vectors needed

In [ ]:
## This line of code for getting all words bugs me
total_vocabulary = set(word for tweet in data_lower for word in tweet)
len(total_vocabulary)

In [ ]:
glove = {}
with open(glove_file,'rb') as f:#'glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [ ]:
glove['republican']

## Using Embeddings in Classification Models - sci-kit learn

- Embeddings can be used in Artificial Neural Networks as an input Embedding Layer
- Embeddings can be used in sci-kit learn models by taking the mean vector of a text/document and using the mean vector as the input into the model. 

## Make Finding Trump Dataset

- At the beginning of his presidency, Trump continued to use a non-secure Android phone for his personal use.
- Tweets from this period can be attributed to Trump or his staffers based on if it came from an Android device or an iPhone

In [ ]:
df = pd.read_csv("../trump_tweets_12012016_to_01012020.csv",
                parse_dates=['created_at'],index_col='created_at')#'https://raw.githubusercontent.com/jirvingphd/capstone-project-using-trumps-tweets-to-predict-stock-market/master/data/trump_tweets_12012016_to_01012020.csv')
# df['datetime'] = pd.to_datetime(df['created_at'])
df = df.sort_index()#.set_index('datetime')
df

In [ ]:
devices = ['Twitter for Android','Twitter for iPhone']
print(f'The first and last timestamps for the group {devices[0]}are:')
start,end= df.groupby('source').get_group(devices[0]).index[[0,-1]]
print(start) ,print(end)

In [ ]:
## Puttig it all together
df_data = df[df['source'].isin(devices)].loc[start:end].copy()
df_data['source'].value_counts(normalize=True)

In [ ]:
undersample = False

if undersample:
    ## Undersampling to match class
    iphone = df_data.loc[df_data['source']=='Twitter for iPhone']
    android = df_data.loc[df_data['source']=='Twitter for Android']
    print(len(iphone),len(android))
    df = pd.concat([iphone, 
                    android.sample(n=len(iphone),
                                   random_state=123)],
                    axis=0)
    print('Data has been undersampled to match mintority class.')
else:
    print('No resampling was done.')
    df= df_data
    
df

In [ ]:
# # df.to_csv('datasets/trump_tweets_iphone_vs_twitter.csv',index=False)
# df= pd.read_csv('datasets/trump_tweets_iphone_vs_twitter.csv')
# df

In [ ]:
df['source'].value_counts()

### Creating Mean Embeddings

In [ ]:
from sklearn.model_selection import train_test_split
from nltk import word_tokenize

y = pd.get_dummies(df['source'],drop_first=True).values
X = df['text'].str.lower().map(word_tokenize)

X_idx = list(range(len(X)))
train_idx,test_idx = train_test_split(X_idx,random_state=123)

X[train_idx]

In [ ]:
def train_test_split_idx(X, y, train_idx,test_idx):
    # try count vectorized first
    X_train = X[train_idx].copy()
    y_train = y[train_idx].copy()
    X_test = X[train_idx].copy()
    y_test = y[train_idx].copy()
    return X_train, X_test,y_train, y_test

X_train, X_test,y_train, y_test = train_test_split_idx(X,y,train_idx,test_idx)

In [ ]:
# df['combined_text'] = df['headline'] + ' ' + df['short_description']
# data = df['combined_text'].map(word_tokenize).values

In [ ]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # Takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # it can't be used in a scikit-learn pipeline  
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

In [ ]:
# target = df['source']
# data = df['text'].map(word_tokenize)
# data_lower = list(map(lambda x: [w.lower() for w in x],data))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Random Forest', RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

models = [('Random Forest', rf),
          ('Support Vector Machine', svc),
          ('Logistic Regression', lr)]
# models = {'Random Forest':RandomForestClassifier(n_estimators=100, verbose=True),
#           'SVC':SVC(),'lr':LogisticRegression()}

In [ ]:
scores = [(name, cross_val_score(model, X_train, y_train, cv=2).mean()) for name, model, in models]

In [ ]:
scores

## Using Embedding Layers in ANN

In [ ]:
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Sequential
from keras import initializers, regularizers, constraints, optimizers, layers


from keras.preprocessing import text, sequence
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences


In [ ]:
import os,sys
sys.path.append("../")
import keras_gridsearch as kg

In [ ]:
from keras.preprocessing import text,sequence
from sklearn.model_selection import train_test_split
from nltk import word_tokenize
from gensim.utils import simple_preprocess
from sklearn import metrics

# y_t = pd.get_dummies(df['source'])#.values
# y_t=y_t['Twitter for Android'].values
y_t = to_categorical(df['source'].map({'Twitter for Android':1,
                                       'Twitter for iPhone':0}),num_classes=2)

# y_t.name='True Trump'

# print(y_t.name)

In [ ]:
MAX_WORDS = 25000
MAX_SEQUENCE_LENGTH = 50

tokenizer = text.Tokenizer(num_words=MAX_WORDS)

tokenizer.fit_on_texts(X) #df['text'])
sequences = tokenizer.texts_to_sequences(X) #df['text'])

X_t = sequence.pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_t.shape

In [ ]:
X_t[-1]

In [ ]:
X_train, X_test, y_train, y_test =train_test_split(X_t,y_t,random_state=123) 
X_train.shape,y_test.shape
# pd.Series(y_test).value_counts(normalize=True)
y_test.shape
X_test

In [ ]:
EMBEDDING_SIZE = 128 #where codealong get this?

embedding_matrix = np.zeros((len(total_vocabulary) + 1, EMBEDDING_SIZE))
for word, i in enumerate(total_vocabulary):#.items():
    embedding_vector = glove.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        
        
embedding_layer = Embedding(len(total_vocabulary) + 1,
                            EMBEDDING_SIZE,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=True)

In [ ]:
# EMBEDDING_SIZE = 128 #where codealong get this?

model=Sequential()
model.add(Embedding(MAX_WORDS, EMBEDDING_SIZE))
model.add(LSTM(25,return_sequences=True))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.5))
model.add(Dense(25, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',#'categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2)

y_hat_test = model.predict_classes(X_test)
# print(pd.Series(y_hat_test).value_counts())
kg.evaluate_model(y_test,y_hat_test,history)

In [ ]:
# model = Sequential()
# model.add(Embedding(MAX_WORDS, EMBEDDING_SIZE))
# model.add(LSTM(output_dim=256, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
# model.add(Dropout(0.5))
# model.add(LSTM(output_dim=256, activation='sigmoid', recurrent_activation='hard_sigmoid'))
# model.add(Dropout(0.5))
# model.add(Dense(1, activation='sigmoid'))

In [ ]:
y_hat_test

## RNN or GRU

In [ ]:
## GRU Model
from keras import models, layers, optimizers, regularizers
modelG = models.Sequential()

## Get and add embedding_layer
# embedding_layer = ji.make_keras_embedding_layer(wv, X_train)
modelG.add(Embedding(MAX_WORDS, EMBEDDING_SIZE))

# modelG.add(layers.SpatialDropout1D(0.5))
# modelG.add(layers.Bidirectional(layers.GRU(units=100, dropout=0.5, recurrent_dropout=0.2,return_sequences=True)))
modelG.add(layers.Bidirectional(layers.GRU(units=100, dropout=0.5, recurrent_dropout=0.2)))
modelG.add(layers.Dense(2, activation='softmax'))

modelG.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['acc'])#,'val_acc'])#, callbacks=callbacks)
modelG.summary()

In [ ]:

history = modelG.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

y_hat_test = modelG.predict_classes(X_test)
kg.evaluate_model(y_test,y_hat_test,history)

# APPENDIX

## scikit-learn vectorization
WIP 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
count_vectorizer = CountVectorizer()
tf_transformer = TfidfTransformer(use_idf=False)#TfidfTransformer()
tfidf_transformer = TfidfTransformer(use_idf=True)#TfidfTransformer()

In [ ]:
from sklearn.preprocessing import LabelEncoder
le= LabelEncoder()
y = le.fit_transform(df['source'])
y.shape

In [ ]:
X = count_vectorizer.fit_transform(df['text'])
X_tf = tf_transformer.fit_transform(X)
X_tfidf = tfidf_transformer.fit_transform(X)

In [ ]:
X_tf.shape,X_tfidf.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_idx = list(range(X.shape[0]))
train_idx,test_idx = train_test_split(X_idx,random_state=123)


def train_test_split_idx(X, y, train_idx,test_idx):
    # try count vectorized first
    X_train = X[train_idx].copy()
    y_train = y[train_idx].copy()
    X_test = X[train_idx].copy()
    y_test = y[train_idx].copy()
    return X_train, X_test,y_train, y_test

X_dict = {'count':X_tf,
         'tfidf':X_tfidf}

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import cross_val_score



# svc = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
#                 ('Support Vector Machine', SVC())])

# lr = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
#               ('Logistic Regression', LogisticRegression())])

# models = {'Random Forest':RandomForestClassifier(n_estimators=100, verbose=True),
#           'SVC':SVC(),'lr':LogisticRegression()}

In [ ]:
# res = [['Method','Model',"Result"]]

# for tf_type,X_data in X_dict.items():
#     X_train, X_test,y_train, y_test = train_test_split_idx(X_data,y,train_idx,test_idx)
    
#     for name, model in models.items():
    
# #     rf = RandomForestClassifier(n_estimators=100,verbose=True)
#         cv_res = cross_val_score(model, X_train,y_train, cv=5)
#         res.append([tf_type,name,cv_res.mean()])

# pd.DataFrame(res[1:],columns=res[0]).sort_values("Result",ascending=False)

In [ ]:


# rf_params = dict(n_estimators=100, verbose=True)

# rf =Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
#               ('Random Forest',RandomForestClassifier(**rf_params))])

# svc = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
#                 ('Support Vector Machine', SVC())])

# lr = Pipeline([('Word2Vec Vectorizer', W2vVectorizer(glove)),
#               ('Logistic Regression', LogisticRegression())])

In [ ]:
# models = [('Random Forest', rf),
#           ('Support Vector Machine', svc),
#           ('Logistic Regression', lr)]

In [ ]:
# # res = [['Model','Score']]
# res=[['Model','Scores']]
# for (name, model) in models:
#     print(name)
#     cv_res = cross_val_score(model, data_lower, df['source'], cv=5).mean()
#     res.append([name,cv_res])
    
# pd.DataFrame(res[1:],columns=res[0])

## Practicing Text Preprocessing with Trump's Tweets

## LAST CLASS

### Removing Stopwords

In [ ]:
## Make a list of stopwords to remove
from nltk.corpus import stopwords
import string

In [ ]:
# Get all the stop words in the English language
stopwords_list = stopwords.words('english')
stopwords_list+=string.punctuation
print(stopwords_list)
stopwords_list.remove('until')
stopwords_list.extend(['“','...','”'])

In [ ]:
## Commentary on not always accepting what is or isn't in stopwords
'until' in stopwords_list

In [ ]:
stopped_tokens = [w.lower() for w in tokens if w.lower() not in stopwords_list]
freq = FreqDist(stopped_tokens)
freq.most_common(100)

In [ ]:
from nltk import word_tokenize
from ipywidgets import interact

@interact
def tokenize_tweet(i=(0,len(corpus)-1)):
    from nltk.corpus import stopwords
    import string
    from nltk import word_tokenize,regexp_tokenize
    
    print(f"- Tweet #{i}:\n")
    print(corpus[i],'\n')
    tokens = word_tokenize(corpus[i])

    # Get all the stop words in the English language
    stopwords_list = stopwords.words('english')
    stopwords_list += string.punctuation
    stopped_tokens = [w.lower() for w in tokens if w not in stopwords_list]
    
    print(tokens,end='\n\n')
    print(stopped_tokens)

In [ ]:
## Get FreqDist for Cleaned Text Data
corpus[:20]

### Comparing Phases of Proprocessing/Tokenization

In [ ]:
# def clean_text(text,exclude_words=['until']):
#     from nltk.corpus import stopwords
#     import string
#     from nltk import word_tokenize,regexp_tokenize
#     ## tokenize text
#     tokens = word_tokenize(text)
#     # Get all the stop words in the English language
#     stopwords_list = stopwords.words('english')
#     stopwords_list += string.punctuation
#     stopped_tokens = [w.lower() for w in tokens if w not in stopwords_list]
#     return stopped_tokens

In [ ]:
from nltk import word_tokenize
from ipywidgets import interact

@interact
def tokenize_tweet(i=(0,len(corpus)-1)):
    from nltk.corpus import stopwords
    import string
    from nltk import word_tokenize,regexp_tokenize
    
    print(f"- Tweet #{i}:\n")
    print(corpus[i],'\n')
    tokens = word_tokenize(corpus[i])

    # Get all the stop words in the English language
    stopwords_list = stopwords.words('english')
    stopwords_list += string.punctuation
    stopped_tokens = [w.lower() for w in tokens if w not in stopwords_list]
    
    print(tokens,end='\n\n')
    print(stopped_tokens)

## Regular Expressions

- Best regexp resource and tester: https://regex101.com/

    - Make sure to check "Python" under Flavor menu on left side.

In [ ]:
text =  corpus[6615]
text

In [ ]:
text2=corpus[7347]
text2

In [ ]:
from nltk import regexp_tokenize
pattern = r"([a-zA-Z]+(?:'[a-z]+)?)"
regexp_tokenize(text,pattern)

In [ ]:
print('[i] Word Tokenize:',end='\n'+'---'*20+'\n')
print(word_tokenize(text))

print('\n[i] Regexp Tokenize:',end='\n'+'---'*20+'\n')
print(regexp_tokenize(text,pattern))

In [ ]:
def clean_text(text,regex=True):
    from nltk.corpus import stopwords
    import string
    from nltk import word_tokenize,regexp_tokenize

    ## tokenize text
    if regex:
        pattern = r"([a-zA-Z]+(?:'[a-z]+)?)"
        tokens= regexp_tokenize(text,pattern)
    else:
        tokens = word_tokenize(text)
    # Get all the stop words in the English language
    stopwords_list = stopwords.words('english')
    stopwords_list += string.punctuation
    stopped_tokens = [w.lower() for w in tokens if w not in stopwords_list]
    return stopped_tokens

In [ ]:
# @interact
# def regexp_tokenize_tweet(i=(0,len(corpus)-1)):
#     print(f"- Tweet #{i}:\n")
#     print(corpus[i],'\n')
#     from nltk import regexp_tokenize
#     pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
#     tokens= regexp_tokenize(corpus[i],pattern)

#     # It is usually a good idea to lowercase all tokens during this step, as well
#     stopped_tokens = [w.lower() for w in tokens if w not in stopwords_list]
#     print(tokens,end='\n\n')
#     return print(stopped_tokens)

In [ ]:
import re

def find_urls(string): 
    return re.findall(r"(http[s]?://\w*\.\w*/+\w+)",string)

def find_hashtags(string):
    return re.findall(r'\#\w*',string)

def find_retweets(string):
    return re.findall(r'RT [@]?\w*:',string)

def find_mentions(string):
    return re.findall(r'\@\w*',string)

In [ ]:
find_urls(text)

In [ ]:
find_mentions(text2)

### Stemming/Lemmatization

In [ ]:

from nltk.stem.wordnet import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print(lemmatizer.lemmatize('feet')) # foot
print(lemmatizer.lemmatize('running')) # run [?!] Does not match expected output

In [ ]:
text_in =  corpus[6615]

# # urls = find_urls(text)
# def clean_text(text,regex=True):
#     from nltk.corpus import stopwords
#     import string
#     from nltk import word_tokenize,regexp_tokenize

#     ## tokenize text
#     if regex:
#         pattern = r"([a-zA-Z]+(?:'[a-z]+)?)"
#         tokens= regexp_tokenize(text,pattern)
#     else:
#         tokens = word_tokenize(text)
#     # Get all the stop words in the English language
#     stopwords_list = stopwords.words('english')
#     stopwords_list += string.punctuation
#     stopped_tokens = [w.lower() for w in tokens if w not in stopwords_list]
#     return stopped_tokens

def process_tweet(text,as_lemmas=False,as_tokens=True):
#     text=text.copy()
    for x in find_urls(text):
        text = text.replace(x,'')
        
    for x in find_retweets(text):
        text = text.replace(x,'')    
        
    for x in find_hashtags(text):
        text = text.replace(x,'')    

    if as_lemmas:
        from nltk.stem.wordnet import WordNetLemmatizer
        lemmatizer = WordNetLemmatizer()
        text = lemmatizer.lemmatize(text)
    
    if as_tokens:
        text = clean_text(text)
    
    if len(text)==0:
        text=''
            
    return text

In [ ]:
@interact
def show_processed_text(i=(0,len(corpus)-1)):
    text_in = corpus[i]#.copy()
    print(text_in)
    text_out = process_tweet(text_in)
    print(text_out)
    text_out2 = process_tweet(text_in,as_lemmas=True)
    print(text_out2)

In [ ]:
corpus[:6]

## Text Classification

> Potential Tasks: Classify Android vs iPhone tweets (from period where Android tweets still exist

In [ ]:
df['datetime'] = pd.to_datetime(df['created_at'])
df

df = df.set_index('datetime').sort_index()
df

In [ ]:
df['clean_text'] = df['text'].apply(process_tweet)
df

In [ ]:
android = df.groupby('source').get_group('Twitter for Android')
android.index

iphone = df.groupby('source').get_group('Twitter for iPhone').loc[:android.index[-1]]
iphone

In [ ]:
len(android), len(iphone)

In [ ]:
df_corpus = pd.concat([iphone,android],axis=0)
df_corpus['source'].value_counts()

### Vectorization 

- Count vectorization
- Term Frequency-Inverse Document Frequency (TF-IDF)
    -  Used for multiple texts
    
    
**_Term Frequency_** is calculated with the following formula:

$$ \text{Term Frequency}(t) = \frac{\text{number of times it appears in a document}} {\text{total number of terms in the document}} $$ 

**_Inverse Document Frequency_** is calculated with the following formula:

$$ IDF(t) = log_e(\frac{\text{Total Number of Documents}}{\text{Number of Documents with it in it}})$$

The **_TF-IDF_** value for a given word in a given document is just found by multiplying the two!


## Questions/Topics 
- Next time: vectorization
- Vs Embeddings

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [ ]:
vectorizer.fit_transform(df_corpus['clean_text'].values)